Kyle Kraft ME 480 HW 5

Setup Code

In [36]:
from solid import *
from solid.utils import *
import viewscad
import numpy as np

r = viewscad.Renderer(openscad_exec='C:\\Program Files\OpenSCAD\openscad.exe')

Part 1

In [21]:
def techbeam(x,y):
    Lc2c = 8*(x-1) #center to center length
    Wc2c = 8*(y-1) #center to center width
    h = 7.76
    rad = 7.38/2 #rad of outside of ends of beam
    innerrad = 5/2
    x0y0 = cylinder(rad,h)
    x1y0 = translate([Lc2c,0,0])(x0y0)
    x0y1 = translate([0,Wc2c,0])(x0y0)
    x1y1 = translate([Lc2c,Wc2c,0])(x0y0)
    beam = hull()(x0y0,x1y0,x0y1,x1y1)
    hole = cylinder(innerrad,h)
    holes = 0
    for i in range(x):
        for j in range(y):
            #print(i,j)
            holes = holes + translate([i*8,j*8,0])(hole)
    beam = beam - holes
    r.render(beam)

In [27]:
techbeam(2,3)

BONUS!! for part 1)

In [52]:
def techbeambonus(x,y,crossedholes):
    Lc2c = 8*(x-1) #center to center length
    Wc2c = 8*(y-1) #center to center width
    h = 7.76
    rad = 7.38/2 #rad of outside of ends of beam
    innerrad = 5/2
    x0y0 = cylinder(rad,h)
    x1y0 = translate([Lc2c,0,0])(x0y0)
    x0y1 = translate([0,Wc2c,0])(x0y0)
    x1y1 = translate([Lc2c,Wc2c,0])(x0y0)
    beam = hull()(x0y0,x1y0,x0y1,x1y1)
    hole = cylinder(innerrad,h)
    crosshole = cube([5,2,h+.1],True) + cube([2,5,h+.1],True)
    holes = 0
    for i in range(x):
        for j in range(y):
            #print(i,j)
            if crossedholes[i,j] == 0:
                holes = holes + translate([i*8,j*8,0])(hole)
            if crossedholes[i,j] == 1:
                holes = holes + translate([i*8,j*8,h/2])(crosshole)
    beam = beam - holes
    r.render(beam)

In [54]:
crossedholes = np.array([[0,0,1,0,0], #schematic of beam with hole types
                         [0,0,0,1,1]]) #0 means normal hole, 1 means crossed hole
techbeambonus(2,5,crossedholes)

Part 2

In [134]:
#define a table of nema specs
#each line is a motor size
#a line is [NEMA size, bolt hole dist, shaft dia, pilot dia, bolt size]
#all dimensions in mm
nematable = np.array([[8, 16, 4, 15, 3],
                      [11, 23, 5, 22, 4],
                      [14, 26, 5, 22, 4],
                      [17, 31, 5, 22, 3.26],
                      [23, 47.14, 6.35, 38.1, 5],
                      [34, 69.7, 9.5, 73, 5.5],
                      [42, 88.9, 16, 55.5, 7.14]])

def nemamount(nema):
    if nema == 8:
        line = 0
    elif nema == 11:
        line = 1
    elif nema == 14:
        line = 2
    elif nema == 17:
        line = 3
    elif nema == 23:
        line = 4
    elif nema == 34:
        line = 5
    elif nema == 42:
        line = 6
    #print(line)
    bhc2c = nematable[line,1] #bolt hole center to center dist
    shaft = nematable[line,2] #shaft dia
    pilot = 1 + nematable[line,3] #pilot dia plus a little margin
    bhdia = nematable[line,4] #bolt hole size
    
    #making base
    h = 7
    outerrad = bhdia/2 + 3
    x0y0 = cylinder(outerrad,h)
    x1y0 = translate([bhc2c,0,0])(x0y0)
    x0y1 = translate([0,bhc2c,0])(x0y0)
    x1y1 = translate([bhc2c,bhc2c,0])(x0y0)
    boss = hull()(x0y0,x1y0,x0y1,x1y1)
    #making pilot hole
    pilot = translate([bhc2c/2,bhc2c/2,0])(cylinder(pilot/2,h))
    #making bolt holes
    hole = cylinder(bhdia/2,h)
    holes = hole
    holes = holes + translate([bhc2c,0,0])(hole)
    holes = holes + translate([0,bhc2c,0])(hole)
    holes = holes + translate([bhc2c,bhc2c,0])(hole)
    #making technic mounting posts
    post = translate([0,0,h/2])(rotate(90,[0,1,0])(cylinder(4.8/2,7.76)))
    posts = translate([-outerrad-7,0,0])(post)
    #adding a second post if space permits: fits on all but NEMA 11
    if nema != 11:
        posts = posts + translate([0,8,0])(posts)
    #need a coord transform to use mirror function for posts
    posts = translate([-bhc2c/2,-bhc2c/2,0])(posts)
    posts = posts + mirror([1,0,0])(posts)
    posts = posts + mirror([0,1,0])(posts)
    posts = translate([bhc2c/2,bhc2c/2,0])(posts)
    #coord transform undone for posts
    final = boss - pilot - holes + posts
    r.render(final)

In [135]:
nemamount(34)


Part 3

In [ ]:
def shaftadapter(size):
    